In [ ]:
# Sentence Vecの作り方（2022.12月少し修正版）
import math
import statistics

In [ ]:
!pip install janome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 40.3 MB/s eta 0:00:00


In [ ]:
#ここで、分析対象テキストファイルをアップロードします。

In [ ]:
#形態素解析をしたファイルを作成
from janome.tokenizer import Tokenizer
def split_text(src, out):
    t = Tokenizer()
    with open(src, encoding='utf8') as fin:
        with open(out, mode='w', encoding='utf8') as fout:
            for line in fin:
              tokens = t.tokenize(line)
              for token in tokens:
                fout.write('%s\n' % token)


In [ ]:
# 自分のファイルに変更する時は以下のutf8gingatetsudono_yoru.txtを変更する。
split_text('/content/text.txt','tsv.out')


FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!head -50 tsv.out

In [ ]:
#形態素解析した結果ファイルを使い、形態素スペース区切りの本文テキストを出力する
def sepfile_from(src,out):
  with open(src, encoding='utf8') as fin:
    with open(out, mode = 'w', encoding='utf8') as fout:
      words = []
      for line in fin:
        a = line.split('\t')
        # 語の出現形だけでなく、品詞も考慮
        pos = a[1].split(',')
        words.append(a[0]+"-"+pos[0])
        if a[0] == u'。':
          #ファイルの最後にも「。」がないと処理しない
          fout.write('%s\n' % ' '.join(words))
          words = []

sepfile_from('/content/tsv.out','sent.txt')


In [ ]:
!head -50 sent.txt

In [ ]:
#形態素スペース区切り本文ファイルの語彙リストを作る
from collections import Counter
def count_vocabdist(src,out):
  with open(src, encoding='utf8') as fin:
    text = fin.read()
    #split関数は引数なしの時は、改行、タブ、スペース等を区切り記号にみなしてくれる。
    words = text.split()
    wdist = Counter(words)
  with open(out,mode='w', encoding='utf8') as fout:
      for k in wdist:
        hinshi= k.split('-')[-1]
        word=k.split('-')[0]
        fout.write('%s\t%s\n' % (k,wdist[k]))

def count_vocabdist2(src,out):
  with open(src, encoding='utf8') as fin:
    text = fin.read()
    #split関数は引数なしの時は、改行、タブ、スペース等を区切り記号にみなしてくれる。
    words = text.split()
    wdist = Counter(words)
  with open(out,mode='w', encoding='utf8') as fout:
      for k in wdist:
        hinshi= k.split('-')[-1]
        word=k.split('-')[0]
        if (hinshi=="名詞" or hinshi=="形容詞") and wdist[k]>6 and (word!='(' and word!=')' and word!='こと'and word!='これ')and len(word)>1:
          fout.write('%s\t%s\n' % (k,wdist[k]))

count_vocabdist('/content/sent.txt','vocadist.tsv')
count_vocabdist2('/content/sent.txt','vocadist2.tsv')


In [ ]:
!head -50 vocadist2.tsv

In [ ]:
# ここでvocadist.tsvをダウンロードし、
# google spreadsheetなりで、対象語リストを作成してください
# 作成した対象語リストはvocadist2.tsvとしてください。

In [ ]:
#スペース区切り本文テキストと語彙リストから文ベクトルを作る
def make_sentvec(src, voca, out):
  vocabhash={}
  vocablist=[]
  with open(voca, encoding='utf8') as fv:
    for text in fv:
      a = text.split()
      vocablist.append(a[0])
      vocabhash[a[0]]=len(vocabhash)
  #
  fo = open(out, mode='w', encoding='utf8')
  fo.write('S#')
  for w in vocablist:
    fo.write('\t%s' % w)
  fo.write('\n')
  # 
  sno = 0
  with open(src, encoding='utf8') as fin:
    for line in fin:
      vec = [0]*len(vocabhash)
      a = line.split()
      for w in a:
        if w in vocabhash:
          vec[vocabhash[w]] +=1
      fo.write('%d' % sno)
      for d in vec:
        fo.write('\t%d' % d)
      fo.write('\n')
      sno+=1
  fo.close()

make_sentvec('sent.txt','vocadist2.tsv','sentvec.txt')


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#tsvの文ベクトルファイルをデータフレームとして読み込む
df = pd.read_table('sentvec.txt')

In [ ]:
#1列目の行番号フィールド列を削除する。
df2 = df.drop('S#',axis=1)

In [ ]:
#データフレームの列名を配列にする:names。
#語とidとの辞書を作る: wordiddic
names = df2.columns.values
wordiddic ={}
for w in names:
  wordiddic[w] = len(wordiddic)


In [ ]:
#numpyの2次元配列に変換する。
#
mata = df2.to_numpy()
mata


In [ ]:
#転置行列に元の行列を掛ける
#ある種の語と語と間の相関行列が計算される
corrmat= np.matmul(mata.T,mata)
corrmat

In [ ]:
#単語の共起がある:1 か、ない:0だけで示す共起行列を作る
r, c = corrmat.shape
corrmat1=np.zeros((r,c))
#原始的なループですみません、、、
for i in range(r):
  for j in range(c):
    corrmat1[i][j]=1 if corrmat[i][j] > 0 else 0


In [ ]:
weights={}
with open("vocadist2.tsv", encoding='utf8') as f:
  for txt in f:
    name=txt.split()[0]
    weight=txt.split()[-1]
    weights[name]=int(weight)
print(weights)


In [ ]:
from IPython.core.display import Math
#任意の語に対するすべての語のユークリッド距離の算出

list_f=[]

for n in names:
  target = n
  simlist = {}
  for w in names:
    simlist[w] = np.sqrt(np.power(corrmat1[wordiddic[target]]-corrmat1[wordiddic[w]],2).sum())  
  #昇順にソートして表示
  a= sorted(simlist.items(),key=lambda x:x[1])
  for i in range(0,10):
    list_f.append((n,a[-1-i],a[-1-i][-1]))

tmp_f=sorted(list_f,key=lambda x:x[-1],reverse=True)
result_f= []
for i,f in zip(range(len(tmp_f)),tmp_f):
  result_f.append((f[0],f[1][0],f[-1]))

print("対照",result_f,sep='\n')
print("ベクトルの次元",len(names),sep="\n")

  



In [ ]:
from IPython.core.display import Math
#任意の語に対するすべての語のユークリッド距離の算出

list_f=[]

for n in names:
  target = n
  simlist = {}
  for w in names:
    simlist[w] = np.dot(corrmat1[wordiddic[target]],corrmat1[wordiddic[w]])
  tmp=simlist.items
  #昇順にソートして表示
  a= sorted(simlist.items(),key=lambda x:x[1])
  for item in a:
    if item[-1]<0:
      list_f.append((n,item,item[-1]))

tmp_f=sorted(list_f,key=lambda x:x[-1],reverse=False)
result_f= []
for i,f in zip(range(len(tmp_f)),tmp_f):
  result_f.append((f[0],f[1][0],f[-1]))

print("対照",result_f,sep='\n')
print("ベクトルの次元",len(names),sep="\n")

  



NameError: ignored

In [ ]:
#
# Part I(準備編）
#

#ベクトル辞書のダウンロード
!wget https://sudachi.s3-ap-northeast-1.amazonaws.com/chive/chive-1.2-mc30_gensim.tar.gz
#ベクトル辞書の定義
import gensim
vecdic = "/content/chive-1.2-mc30_gensim/chive-1.2-mc30.kv"

vectors = gensim.models.KeyedVectors.load(vecdic)



In [ ]:
#対照性の実験
print(vectors.most_similar(positive=["アンドロイド","バックアップ"], negative=["グーグル"],topn=5))

[('リストア', 0.5200445055961609), ('フルバックアップ', 0.4968959093093872), ('backup', 0.4944484233856201), ('ックアップ', 0.4928125739097595), ('trueimage', 0.48917585611343384)]
